<a href="https://colab.research.google.com/github/a-forty-two/EY_batch11_3Mar/blob/main/5Mar_001_llm_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color="#76b900">**Notebook 1:** Getting Started With Large Language Models</font>

**Welcome To The Course!** This is the first content notebook and is intended to springboard you into the LLM loading workflow with some insights about our problem, our resources, and our objectives!

#### **Learning Objectives:**

- Review some basic assumptions about deep learning and show how they extend to language modeling.
- Pull in your first LLM into the environment, investigate its architecture, and see how it performs!

-------

<br>

## **Part 1.1:** Recalling Deep Learning

Throughout your learning adventure with deep learning, you have probably optimized a variety of models for tasks like classification and regression. In order, you probably advanced in something like the following:

- When you started out, you used **linear and logistic regression** to model and interpret simple linear relationships that associated your inputs with your outputs.
- When that wasn't enough, you started **stacking linear layers one after another and adding non-linear activations** to give your model more predictive power.
- When your data started getting intractably high-dimensional, you started using more **informed sparsely-connected techniques like convolution** to add more control to your reasoning criteria.
- When you realized that you didn't have enough data to properly train your models for each specific task, you got **pre-trained components (i.e. VGG-16/ResNet)** that were trained on a giant repository of training data and already contained the necessary logic you wanted.

> <div><img src="imgs/machine-learning-process.jpg" width="800"/></div>
>
> **Source: [High-Performance Data Science with RAPIDS | NVIDIA](https://www.nvidia.com/en-us/deep-learning-ai/software/rapids/)**

If you’ve gone through these steps, you already possess the foundational skills to tackle complex topics, including the vast field of language modeling.

Like vision, language is highly complex and high-dimensional. For instance, a simple 200x200 color image contains $200\times 200\times 3 = 120,000$ features! Now, consider the even larger number of combinations possible in natural language—**it’s enormous!** Fortunately, creative techniques and large pre-trained models simplify the problem, providing efficient solutions.

**This course will show you how to approach language modeling, the tools available, and the types of problems you can solve!**

-------

<br>

## **Part 1.2:** Pulling In Our First LLM

Rather than building models from scratch, this course focuses on using powerful tools to simplify the process. A great place to start is **HuggingFace** &#x1F917;.

[**HuggingFace**](https://huggingface.co/) is an open-source community that offers simple strategies for accessing, developing, and hosting large deep learning models for testing and deployment. The topics they support span many tasks and modalities, but we'll be focusing on large language models (**LLMs**) for most of this course.

When searching through the [**HuggingFace Models catalog**](https://huggingface.co/models?sort=downloads&search=bert), you'll quickly stumble upon the [`bert-base-uncased`](https://huggingface.co/bert-base-uncased) model. Taking a look at its card, you'll see several interesting things:

- **The [`transformers`](https://github.com/huggingface/transformers) Package**: Loading in the model requires the use of the [`transformers`](https://github.com/huggingface/transformers) package, which is HuggingFace’s primary library for language models. Its name refers to the transformer architecture, which we’ll explore further in upcoming sections. You’ll be using `transformers` throughout this course, so feel free to dive into the source code as you go.

- **Pipelines**: HuggingFace simplifies complex deep learning tasks with its [Pipeline API](https://huggingface.co/docs/transformers/main_classes/pipelines). A pipeline allows you to move from input to output without worrying about the internal workings. We’ll demonstrate this with the `bert-base-uncased` model for mask-filling.

As a representative example, we can go ahead and pull in the discussed [`bert-base-uncased`](https://huggingface.co/bert-base-uncased) model and test it out:

In [ ]:
from transformers import pipeline

## Loading in the pipeline and predict the mask fill (example from model card)
unmasker = pipeline(
    'fill-mask',
    model='bert-base-uncased',
    device='cuda',  ## Feel free to use GPU. For such a small model, not necessary
)
unmasker("Hello I'm a [MASK] model.")

**And just like that, it works!** You have just obtained an intuitive answer that makes sense with human logic, which makes you almost forget that there is actually a deep learning model calculating probabilities to make this all work.

**But that's what this course is for: to see what's actually going on behind the scenes and know how to use it to make interesting and useful systems.**

<hr>
<br>

## **Part 1.3:** Dissecting The Pipeline

While the pipeline provides a clean interface—taking strings in and returning a dictionary—it doesn’t give us much insight into what’s happening under the hood. Let’s peel back a layer and examine the approximate inner workings:

In [ ]:
from transformers import AutoTokenizer, BertTokenizer, BertModel, FillMaskPipeline, AutoModelForMaskedLM, BertForMaskedLM, BertForPreTraining

from transformers import AutoTokenizer, AutoModel        ## General-purpose fully-automatic
from transformers import AutoModelForMaskedLM            ## Default import for FillMaskPipeline
from transformers import BertTokenizer, BertForMaskedLM  ## Realized components after automatic resolution

class MyMlmPipeline(FillMaskPipeline):
    ## My Masked Language Modeling Pipeline

    ### CASE 0: Construct your pipeline automatically by pulling in the components
    ###   with their respective configs from HuggingFace. Pipeline assumes preprocessing/postprocessing.
    def __init__(self, *args, **kwargs):
        ## The fully-automatic version
        super().__init__(
            tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased'),
            model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased"),
            # model = BertForMaskedLM.from_pretrained("bert-base-uncased"),
            *args, **kwargs  ## <- pass in any extra arguments
        )

    ### CASE 1: Uncomment out the __call__ method to see what data is flowing.
    """
    def __call__(self, string, verbose=True):
        ## Verbose argument just there for our convenience
        input_tensors = self.preprocess(string)
        if verbose: print('\npreprocess outputs:\n', input_tensors, '\n')
        output_tensors = self.forward(input_tensors)
        if verbose: print('forward outputs:\n', output_tensors, '\n')
        output = self.postprocess(output_tensors)
        return output
    """

    ### CASE 2: Uncomment out the manual overrides below to verify the pipeline still works
    """
    def preprocess(self, string):
        string = [string] if isinstance(string, str) else string
        inputs = self.tokenizer(string, return_tensors="pt")           ### strings -> indices
        inputs = {k: v.to("cuda") for k, v in inputs.items()}          ### move to GPU
        return inputs

    def forward(self, tensor_dict):
        output_tensors = self.model.forward(**tensor_dict)             ### indices -> vectors -> probabilities
        return {**output_tensors, **tensor_dict}

    def postprocess(self, tensor_dict):
        ## Very Task-specific; see FillMaskPipeline.postprocess
        tensor_dict = {k: v.to("cpu") for k, v in tensor_dict.items()} ### move off GPU
        return super().postprocess(tensor_dict)                        ### probabilities (or vectors) -> outputs
    """

unmasker = MyMlmPipeline(device="cuda")
unmasker("Hello, Mr. Bert! How is it [MASK]?")

We can also see that the model primarily consists of two core components:
- **Tokenizer**: Converts input strings into a format the model can process.
- **Model**: The deep learning engine that transforms input tensors into output tensors.

Together, these components support the pipeline’s intuitive workflow:
- `preprocess`: human-intuitive input $\to$ tensor inputs. Facilitated by `tokenizer`
- `forward`: tensor inputs $\to$ tensor outputs. Facilitated by `model`
- `postprocess`: tensor outputs $\to$ human-intuitive outputs. Facilitated by the pipeline task.

For deep learning, this flow makes sense. The model operates on numbers, but the users are working with words, to this abstraction greatly simplifies the LLM interface for typical scenarios.

-------

<br>

## **Part 1.4:** Your Course Environment

Since these models are so easy to pull in, we need to establish some limitations! Throughout this course, feel free to explore different models, but be mindful of their size—**your compute environment is powerful, but not limitless.** We’ve pre-loaded several models for you, which you can review (along with their licenses) in the [**extras_and_licenses/99_licenses.ipynb**](extras_and_licenses/99_licenses.ipynb).

For this course, you’ll be using a high-performance compute setup which is sufficient for the exercises and has the following key resources:

- **System Memory**: Large language models can require massive memory, with some needing tens or even hundreds of GB. This environment is equipped to handle significant workloads, but try to keep model sizes reasonable unless otherwise instructed.
  
- **GPU**: GPU acceleration is crucial for fast deep learning, especially given the parallel processing demands of deep models. The thousands of [**CUDA cores**](https://en.wikipedia.org/wiki/CUDA) in modern NVIDIA GPUs significantly speed up training and inference by performing parallelizable tasks (like high-dimensional matrix multiplication) very quickly.
  
    - **GPU RAM**: Large models need to reside in GPU memory for fast inference. Limited GPU RAM can be a bottleneck for using accelerated LLMs, so manage your model sizes accordingly.

Here’s a breakdown of your allocated resources:

In [ ]:
%%bash
echo """
===================================================
GPU SPECIFICATION
==================================================="""
nvidia-smi
echo """
===================================================
MEMORY SPECIFICATION
==================================================="""
free -h

<br>

**So, while you have a solid compute budget, it’s not infinite!**

Before moving to the next notebook, restart the Jupyter kernel using the following code to clear memory and avoid issues with future notebooks:

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)